<a href="https://colab.research.google.com/github/XmYx/stable-diffusion-gradio-anim-opt/blob/main/P_HLKY_Deforum_StableDiffusion_GradIO_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Stable Diffusion GUI v0.4 (3D/2D Anim, Batch, Interpolation, Variation)

requirements:
          - Use the gradio link from the output to get to the GUI.
          - KeyFrame builder added
          - Multitasking Enabled
          - sd-v1-4.ckpt on your Google Drive (Only directory path needed) 
          - sd-clip-vit-l14-img-embed_ema_only.ckpt on your Google Drive (Use Filename as well in the settings)
          - Variations Model is optional, turn the Skip Switch on to skip loading it


Link to model: https://huggingface.co/CompVis/stable-diffusion-v-1-4-original
Link to Variations Model: https://huggingface.co/lambdalabs/stable-diffusion-image-conditioned/blob/main/sd-clip-vit-l14-img-embed_ema_only.ckpt


Please find expanded explanation at GitHub:
https://github.com/XmYx/stable-diffusion-gradio-anim-opt

Features: 8192 x 8192 max res (really slow) and 8x Upscale without running out of memory. Featuring all features of Deforum's Notebook, plus Variations, Inpainting, and Noodle Soup Prompts.


GPU CHECK

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-a33890a0-b99f-fc20-8e93-c56e70f0c690)


#Setup Optimized Anim GUI with Variations and Noodle Soup

In [ ]:

use_ngrok = "" #@param {type:"boolean"}
ngrok_token = "" #@param {type:"string"}


from google.colab import drive
drive.mount('/gdrive')
# 2 - Download Ngrok

if use_ngrok == True:
    ! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    ! unzip -qq -n ngrok-stable-linux-amd64.zip

    # 3 - setup Ngrok - authtoken

    #Ask token
    print("Get your authtoken from https://dashboard.ngrok.com/auth")
    import getpass
    if ngrok_token == "":
        authtoken = getpass.getpass()
    else:
        authtoken = ngrok_token

    Create tunnel
    get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok http 7860 &')

!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install omegaconf==2.2.3 einops==0.4.1 pytorch-lightning==1.7.4 torchmetrics==0.9.3 torchtext==0.13.1 transformers==4.21.2 kornia==0.6.7 gradio
!pip install git+https://github.com/TencentARC/GFPGAN#egg=GFPGAN
%mkdir /content/models
%mkdir /content/models/pretrained
!wget -N https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P /content/models
!wget -N https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt -P /content/models
!wget -N https://cloudflare-ipfs.com/ipfs/Qmd2mMnDLWePKmgfS8m6ntAg4nhV5VkUyAydYBp8cWWeB7/AdaBins_nyu.pt -P /content/models/pretrained
%cd /content
!pip install git+https://github.com/TencentARC/GFPGAN#egg=GFPGAN
!git clone https://github.com/shariqfarooq123/AdaBins.git
!git clone https://github.com/isl-org/MiDaS.git
!git clone https://github.com/MSFTserver/pytorch3d-lite.git
!wget -N https://raw.githubusercontent.com/XmYx/waifu-diffusion-gradio-hosted-by-colab-en/main/anim/gradio_anim_01.py -P /content/stable-diffusion/scripts
!git clone https://github.com/deforum/k-diffusion/
!pip install -e /content/k-diffusion
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install accelerate ftfy jsonmerge matplotlib resize-right timm torchdiffeq
!pip install -e git+https://github.com/xinntao/Real-ESRGAN#egg=realesrgan
!rm -r /content/stable-diffusion-gradio-anim-opt
%cd /content/
!git clone https://github.com/XmYx/stable-diffusion-gradio-anim-opt
!pip install -e /content/stable-diffusion-gradio-anim-opt
!pip install fire
%mkdir /content/soup
!wget -q --show-progress --no-cache --backups=1 'https://raw.githubusercontent.com/WASasquatch/noodle-soup-prompts/main/nsp_pantry.py' -P /content/soup

In [ ]:
#@markdown **Stable Diffusion Model Directory: (no filename)**

stable_diffusion_models_path = "/gdrive/MyDrive/AI/models/" #@param {type:"string"}

#@markdown **Snapshot Config directory**


cfg_path = "/gdrive/MyDrive/AI/sd_anims/_configs"#@param {type:"string"}


#@markdown **Skip Variations Model (Optional)**

skip_variations = True #@param {type:"boolean"}

#@markdown **Variations Model File (with filename) (Optional)**

variations_model_file = "/gdrive/MyDrive/AI/models/sd-clip-vit-l14-img-embed_ema_only.ckpt" #@param {type:"string"}


if skip_variations == True:
  skip_var = "--no_var True"
else:
  skip_var = f"--var_ckpt {variations_model_file}"

START = f"--ckpt {stable_diffusion_models_path} {skip_var} --cfg_path {cfg_path}"
if use_ngrok == True:
    get_ipython().system_raw('./ngrok http 7860 &')
%cd /content/
!python3 /content/stable-diffusion-gradio-anim-opt/anim/sd_anim_gui_v04.py {START}

In [ ]:
#@markdown **Update Cell**
!rm -r /content/stable-diffusion-gradio-anim-opt
%cd /content
!git clone https://github.com/jmpaz/stable-diffusion-anim-webui
!pip install gradio --upgrade
